In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import glorot_uniform
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers

In [33]:
df=pd.read_csv('fashion-mnist_train.csv')

In [34]:
df.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
x=np.array(df.drop(columns=['label']))
y_actual=np.array(df['label'])
print(y_actual.shape)

(60000,)


In [36]:
y=np.zeros((len(y_actual),10))
for i in range(0,len(y_actual)):
    index=y_actual[i]
    y[i,index]=1

In [37]:
from keras_tuner.tuners import RandomSearch

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2)

In [39]:
import keras_tuner as kt
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam

class Classifier():
    def __init__(self):
        self.tuner = None
        self.model = None
        
    def build_model(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('units_input', min_value=10, max_value=30, step=1), activation='selu', input_dim=784, kernel_initializer=GlorotUniform()))
        
        for i in range(hp.Int('num_layers', 1, 10)):
            model.add(Dropout(0.2))
            model.add(Dense(units=hp.Int(f'units_{i}', min_value=5, max_value=30, step=1), activation='selu', kernel_initializer=GlorotUniform()))
        
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax', kernel_initializer=GlorotUniform()))        
        model.compile(
            optimizer=Adam(),
            loss='categorical_crossentropy'
        )
        return model

    def tune_hyperparameters(self, x, y, max_trials=10, executions_per_trial=1):
        tuner = kt.RandomSearch(
            self.build_model,
            objective='val_loss',
            max_trials=max_trials,
            executions_per_trial=executions_per_trial,
            directory='my_dir2',
            project_name='ANN2'
        )
        
        x_scaled = StandardScaler().fit_transform(x)        
        tuner.search(x_scaled, y, epochs=50, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])        
        self.tuner = tuner
        self.model = tuner.get_best_models(num_models=1)[0]

    def train(self, x, y):
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)        
        early_stopping = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)        
        self.model.fit(x_scaled, y, epochs=500, callbacks=[early_stopping])

    def predict(self, x):
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        return self.model.predict(x_scaled)


In [40]:
c=Classifier()
c.tune_hyperparameters(x_train, y_train, max_trials=10, executions_per_trial=1)
c.train(x_train,y_train)

Reloading Tuner from my_dir2\ANN2\tuner0.json


C:\Users\anasm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\anasm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.4638
Epoch 2/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4588
Epoch 3/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4565
Epoch 4/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4492
Epoch 5/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4437
Epoch 6/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4398
Epoch 7/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4409
Epoch 8/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4374
Epoch 9/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4288
Epoch 10/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4289
Epoch 11/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4350
Epoch 12/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.4311
Epoch 13/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4203
Epoch 14/500
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.4156
Epoch 15/500
15

In [41]:
y_pred=c.predict(x_test)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [42]:
from sklearn.metrics import accuracy_score
print(y_test.shape)
print(y_pred.shape)
y_pred_binary = np.argmax(y_pred, axis=1)
y_test_binary = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred_binary == y_test_binary)
print(f"Accuracy: {accuracy}")

(12000, 10)
(12000, 10)
Accuracy: 0.87575


In [44]:
class_wise_accuracy = {}
for i in range(10):
    class_mask = (y_test_binary == i)
    class_accuracy = accuracy_score(y_test_binary[class_mask], y_pred_binary[class_mask])
    class_wise_accuracy[i] = class_accuracy
    print(f"Class {i} Accuracy: {class_accuracy}")


Class 0 Accuracy: 0.8787878787878788
Class 1 Accuracy: 0.9821717990275527
Class 2 Accuracy: 0.8269230769230769
Class 3 Accuracy: 0.8793103448275862
Class 4 Accuracy: 0.8062913907284768
Class 5 Accuracy: 0.9434599156118143
Class 6 Accuracy: 0.6061151079136691
Class 7 Accuracy: 0.9419191919191919
Class 8 Accuracy: 0.9328968903436988
Class 9 Accuracy: 0.9351481184947958
